1. 모델 설정

In [9]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import os

# OCKRA 모델 정의
class OCKRA(BaseEstimator, ClassifierMixin):
    def __init__(self, n_classifiers=100, n_clusters=10, subsample_ratio=0.63):
        self.n_classifiers = n_classifiers
        self.n_clusters = n_clusters
        self.subsample_ratio = subsample_ratio
        self.ensemble_ = []

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input X must be a Pandas DataFrame.")
        
        self.ensemble_ = []
        for _ in range(self.n_classifiers):
            random_features = np.random.choice(X.columns, size=int(len(X.columns) * self.subsample_ratio), replace=False)
            X_subset = X[random_features]
            kmeans = KMeans(n_clusters=self.n_clusters, init='k-means++', n_init=10, random_state=42).fit(X_subset)
            self.ensemble_.append((kmeans, random_features))
        return self

    def reconstruction_error(self, X):
        if not self.ensemble_:
            raise ValueError("Model is not fitted yet. Please call fit() before reconstruction_error().")

        total_similarity = np.zeros(len(X))  # 총 유사도 점수 초기화
        
        for kmeans, features in self.ensemble_:
            X_subset = X[features]
            distances = kmeans.transform(X_subset).min(axis=1)
            delta_i = np.mean(distances)  # 평균 거리 δi
            similarity = self.similarity_score(distances, delta_i)
            total_similarity += similarity
        
        total_similarity /= self.n_classifiers  # 앙상블 평균 유사도
        return total_similarity

    
    def similarity_score(self, distances, delta_i):
        """
        거리값을 기반으로 유사도 점수를 계산합니다.
        
        Args:
            distances (array): 클러스터 중심과의 거리값들
            delta_i (float): 평균 거리 δi
            
        Returns:
            array: 0~1 범위의 유사도 점수
        """
        scores = np.ones(len(distances))  # 기본값 1 (최대 유사도)
        # δi 이하 → 점수 0.6 초과
        scores[distances <= delta_i] = 0.6 + 0.4 * (1 - (distances[distances <= delta_i] / delta_i))
        
        # δi의 3배 이상 → 점수 0.02 이하
        scores[distances >= 3 * delta_i] = 0.02
        
        # 중간값 → 점수는 선형적으로 조정됨
        mask = (distances > delta_i) & (distances < 3 * delta_i)
        scores[mask] = 0.6 - 0.58 * ((distances[mask] - delta_i) / (3 * delta_i - delta_i))
        
        return scores
   


2. 모델 학습 및 성능평가

In [ ]:
# 성능 평가 코드
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 성능 평가 함수
def evaluate_fold(model, X_test, y_test):
    similarity_scores = model.reconstruction_error(X_test)
    threshold = 0.6  # 논문 기준 임계값
    y_pred = (similarity_scores > threshold).astype(int)
    y_test_numeric = y_test.map({'typical': 1, 'atypical': 0})

    return {
        "Accuracy": accuracy_score(y_test_numeric, y_pred),
        "Precision": precision_score(y_test_numeric, y_pred),
        "Recall": recall_score(y_test_numeric, y_pred),
        "F1 Score": f1_score(y_test_numeric, y_pred),
        "AUC": roc_auc_score(y_test_numeric, similarity_scores)
    }


from itertools import product

# 파라미터 리스트 설정
n_clusters_list = [5, 10, 20, 30, 40, 50]  # 클러스터 수 후보들
n_classifiers_list = [25, 50, 100]  # 분류기 수 후보들
subsample_ratio = 0.63  # 고정값

# 파라미터 조합 생성
parameter_combinations = list(product(n_classifiers_list, n_clusters_list))

# 60초 간격 데이터 추출 함수
def extract_60s_intervals(data, time_column=None):
    """
    60초 간격 데이터를 추출하는 함수
    Args:
        data (DataFrame): 입력 데이터
        time_column (str): 시간 기준 컬럼 (선택적)
    Returns:
        DataFrame: 60번째 간격으로 추출된 데이터
    """
    if time_column:
        data = data.sort_values(time_column)  # 시간 정렬
    return data.iloc[::60, :]  # 60번째마다 데이터 추출

# Five-Fold Cross Validation 학습 및 평가
base_path = "processeed_dataset/User1/FiveFoldCrossValidation"
results = []  # 결과 저장

for n_classifiers, n_clusters in parameter_combinations:
    print(f"\n### Training with n_classifiers={n_classifiers}, n_clusters={n_clusters} ###")
    fold_results = []

    # 모델 학습 및 저장
    for fold_id in range(1, 6):  # Fold1 ~ Fold5
        print(f"Processing Fold {fold_id}...")
        fold_path = f"{base_path}/Fold{fold_id}"
        
        # 데이터 로드
        training_path = f"{fold_path}/training.csv"
        training_data = pd.read_csv(training_path)
        
        # 60초 간격 데이터 추출
        training_data_60s = extract_60s_intervals(training_data)
        X_train = training_data_60s.drop(columns=['ANOM_COND'])
        
        # 모델 학습
        model = OCKRA(n_classifiers=n_classifiers, n_clusters=n_clusters, subsample_ratio=subsample_ratio)
        model.fit(X_train)
        
        # 모델 저장
        model_save_path = f"ockra_model_fold{fold_id}.pkl"
        joblib.dump(model, model_save_path)
        print(f"Model for Fold {fold_id} saved to {model_save_path}")
    
    # 성능 평가
    for fold_id in range(1, 6):  # Fold1 ~ Fold5
        print(f"Evaluating Fold {fold_id}...")
        fold_path = f"{base_path}/Fold{fold_id}"
        
        # 데이터 로드
        testing_path = f"{fold_path}/testing.csv"
        testing_data = pd.read_csv(testing_path)
        
        # 60초 간격 데이터 추출
        testing_data_60s = extract_60s_intervals(testing_data)
        X_test = testing_data_60s.drop(columns=['ANOM_COND'])
        y_test = testing_data_60s['ANOM_COND']
        
        # 해당 폴드 모델 불러오기
        model_load_path = f"ockra_model_fold{fold_id}.pkl"
        model = joblib.load(model_load_path)
        
        # 성능 평가
        fold_result = evaluate_fold(model, X_test, y_test)
        fold_results.append(fold_result)
        print(f"Fold {fold_id} Results: {fold_result}")

    # 평균 결과 저장
    fold_results_df = pd.DataFrame(fold_results)
    avg_result = fold_results_df.mean()
    avg_result['n_classifiers'] = n_classifiers
    avg_result['n_clusters'] = n_clusters
    results.append(avg_result)
    print(f"Average Results for n_classifiers={n_classifiers}, n_clusters={n_clusters}:\n{avg_result}")

# 결과 DataFrame 생성 및 저장
results_df = pd.DataFrame(results)
results_df.to_csv("parameter_search_results.csv", index=False)
print("\nParameter search results saved to 'parameter_search_results.csv'.")

# 최종 결과 출력
print("\nFinal Results:")
print(results_df)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

def visualize_clusters(data, n_clusters):
    """
    K-means++로 클러스터링 결과와 중심점을 시각화하는 함수.
    
    Args:
        data (DataFrame): 입력 데이터 (2개의 주요 특징만 선택)
        n_clusters (int): 클러스터 수
    """
    # K-means++ 클러스터링 수행
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
    kmeans.fit(data)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    # 클러스터 시각화
    plt.figure(figsize=(10, 8))
    for cluster_id in range(n_clusters):
        cluster_data = data[labels == cluster_id]
        plt.scatter(cluster_data[:, 0], cluster_data[:, 1], label=f'Cluster {cluster_id}', alpha=0.6)
    
    # 클러스터 중심 시각화
    plt.scatter(centroids[:, 0], centroids[:, 1], s=200, c='red', marker='X', label='Centroids')
    plt.title(f'K-means++ Clustering with {n_clusters} Clusters', fontsize=16)
    plt.xlabel('Feature 1', fontsize=14)
    plt.ylabel('Feature 2', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()

# 데이터 로드 (예시로 Fold1의 데이터를 사용)
training_path = "processeed_dataset/User1/FiveFoldCrossValidation/Fold1/training.csv"
training_data = pd.read_csv(training_path)

# 주요 2개 특징 선택 (예시로 첫 번째, 두 번째 열 사용)
X_train = training_data.drop(columns=['ANOM_COND']).iloc[:, :2].values

# 클러스터 시각화 (예: 클러스터 수 5)
visualize_clusters(X_train, n_clusters=5)


In [22]:
from scipy.stats import friedmanchisquare
import pandas as pd

data = pd.read_csv('parameter_search_results.csv')

# n_classifiers와 n_clusters 각각에 대한 그룹 생성 (AUC대상)
classifiers_groups = data.groupby('n_classifiers')['AUC'].apply(list)
clusters_groups = data.groupby('n_clusters')['AUC'].apply(list)

# Friedman Test 수행
stat_classifiers, p_classifiers = friedmanchisquare(*classifiers_groups)
stat_clusters, p_clusters = friedmanchisquare(*clusters_groups)

# 결과 출력
print("\nFriedman Test Results for n_classifiers:")
print(f"Test Statistic: {stat_classifiers}, p-value: {p_classifiers}")

print("\nFriedman Test Results for n_clusters:")
print(f"Test Statistic: {stat_clusters}, p-value: {p_clusters}\n")

# p-value 해석
if p_classifiers < 0.05:
    print("n_classifiers에 따른 AUC 값 간에 통계적으로 유의미한 차이가 있습니다.")
else:
    print("n_classifiers에 따른 AUC 값 간에 통계적으로 유의미한 차이가 없습니다.")

if p_clusters < 0.05:
    print("n_clusters에 따른 AUC 값 간에 통계적으로 유의미한 차이가 있습니다.")
else:
    print("n_clusters에 따른 AUC 값 간에 통계적으로 유의미한 차이가 없습니다.")



Friedman Test Results for n_classifiers:
Test Statistic: 1.0, p-value: 0.6065306597126334

Friedman Test Results for n_clusters:
Test Statistic: 10.80952380952381, p-value: 0.055290127801493866

n_classifiers에 따른 AUC 값 간에 통계적으로 유의미한 차이가 없습니다.
n_clusters에 따른 AUC 값 간에 통계적으로 유의미한 차이가 없습니다.


In [32]:
import pandas as pd
from scipy.stats import friedmanchisquare

# CSV 파일 로드
data = pd.read_csv('parameter_search_results.csv')

# 필요한 열 추출
auc = data['AUC']
n_classifiers = data['n_classifiers']
n_clusters = data['n_clusters']

# 프리드만 검정 수행
stat, p = friedmanchisquare(auc, n_classifiers, n_clusters)

# 결과 출력
print(f"Friedman test statistic: {stat}")
print(f"P-value: {p}")

# if p < 0.05:
#     #print("유의미한 차이가 있습니다.")
# else:
#     #print("유의미한 차이가 없습니다.")

Friedman test statistic: 30.78873239436617
P-value: 2.062109429491706e-07
